# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [2]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch


In [3]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 52.1 MB/s 
     |████████████████████████████████| 6.6 MB 41.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [4]:
# loading the dataset

train_df = pd.read_csv('drive/MyDrive/q4_data/train.csv')
test_df = pd.read_csv('drive/MyDrive/q4_data/test.csv')

#dataset understanding
train_df.sample(5)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
140801,f1750615e20b41bd,"""The following was deleted from a DISCUSSION: ...",0,0,0,0,0,0
50302,86789e77a309c00d,{unblock|I understand if my test was taken to ...,0,0,0,0,0,0
28298,4aecd648f6be746e,Links I find to be advertising or not needed \...,0,0,0,0,0,0
53443,8ee42012d621623d,"Come on, unblock me, idiot! I am telling the t...",1,0,1,0,1,0
97891,0bb72a733a084744,"""\n\n Last name \n\nSearches turn out a large ...",0,0,0,0,0,0


In [5]:
# let's see the total rows in train, test data and the numbers for the various categories
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
#print(train_df[cols_target].sum())


Total rows in test is 153164
Total rows in train is 159571


In [6]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [11]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [12]:
#train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, test_X.shape)


(159571,) (153164,)


Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [10]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
#elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [ ]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [ ]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]


Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
#Obtain X_test and X_train 
from tqdm import tqdm

X_train = []
X_test = []

for i in tqdm(X):
    X_train.append(get_use_embedding(i))
for i in tqdm(test_X):
    X_test.append(get_use_embedding(i))

X_train = np.array(X_train)
X_test = np.array(X_test)
np.save('drive/MyDrive/q4_data/X_train', X_train)
np.save('drive/MyDrive/q4_data/X_test', X_test)


In [ ]:
X_train_BERT = []
X_test_BERT = []
from tqdm import tqdm
for i in tqdm(X):
  X_train_BERT.append(get_bert_embedding(i))
for i in tqdm(test_X):
  X_test_BERT.append(get_bert_embedding(i))

In [ ]:
bert_to_memory = []
for i in tqdm(range(len(X_train_BERT))):
  tensor = torch.tensor(X_train_BERT[i], dtype=torch.float32, requires_grad=True).cuda()
  np_c = tensor.detach().cpu().numpy()
  bert_to_memory.append(np_c)
X_train_BERT = np.array(bert_to_memory)
np.save('drive/MyDrive/q4_data/X_train_BERT', X_train_BERT)

bert_to_memory_test = []
for i in tqdm(range(len(X_test_BERT))):
  tensor = torch.tensor(X_test_BERT[i], dtype=torch.float32, requires_grad=True).cuda()
  np_c = tensor.detach().cpu().numpy()
  bert_to_memory_test.append(np_c)
X_test_BERT = np.array(bert_to_memory_test)
np.save('drive/MyDrive/q4_data/X_test_BERT', X_test_BERT)


In [9]:
X_train_USE = np.load('drive/MyDrive/q4_data/X_train.npy')
X_test_USE = np.load('drive/MyDrive/q4_data/X_test.npy')
X_train_BERT = np.load('drive/MyDrive/q4_data/X_train_BERT.npy')
X_test_BERT = np.load('drive/MyDrive/q4_data/X_test_BERT.npy')


In [10]:
y_train = np.array(train_df.drop(['id', 'comment_text'], axis = 1))
y_test = pd.read_csv('drive/MyDrive/q4_data/test_labels.csv')
y_test = np.array(y_test.drop(['id'], axis = 1))
temp = np.array([-1,-1,-1,-1,-1,-1])
index = []
for i in range(len(y_test)):
  if(y_test[i] == temp).all():
    index.append(i)

X_test_USE = np.delete(X_test_USE, index, axis=0)
X_test_BERT = np.delete(X_test_BERT, index, axis=0)
y_test = np.delete(y_test, index, axis=0)

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [13]:
!pip install scikit-multilearn

from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

     |████████████████████████████████| 89 kB 3.8 MB/s 


In [ ]:
# Naive Bayes classifier on USE

classifier = BinaryRelevance(GaussianNB())
classifier.fit(X_train_USE, y_train)
predictions = classifier.predict(X_test_USE)
print('Accuracy:', accuracy_score(y_test, predictions))

Accuracy: 0.7983838194379318


In [ ]:
# Naive Bayes classifier on BERT

classifier = BinaryRelevance(GaussianNB())
classifier.fit(X_train_BERT, y_train)
predictions = classifier.predict(X_test_BERT)
print('Accuracy:', accuracy_score(y_test, predictions))

Accuracy: 0.7384413392103536


In [ ]:
#Support Vector Machines on USE

classifier = BinaryRelevance(LinearSVC(dual=False))
classifier.fit(X_train_USE, y_train)
predictions = classifier.predict(X_test_USE)
print('Accuracy:', accuracy_score(y_test, predictions))

Accuracy: 0.8930257275938603


In [ ]:
#Support Vector Machines on BERT

classifier = BinaryRelevance(LinearSVC(dual=False))
classifier.fit(X_train_BERT, y_train)
predictions = classifier.predict(X_test_BERT)
print('Accuracy:', accuracy_score(y_test, predictions))

Accuracy: 0.8893994810716184


In [ ]:
#MLP on USE

classifier = MLPClassifier(hidden_layer_sizes=(20), activation = 'logistic', solver = 'adam', max_iter= 50, batch_size= 128, random_state = 0)
classifier.fit(X_train_USE, y_train)
predictions = classifier.predict(X_test_USE)
print('Accuracy:', accuracy_score(y_test, predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy: 0.8919472318609521


In [ ]:
#MLP on BERT

classifier = MLPClassifier(hidden_layer_sizes=(20), activation = 'logistic', solver = 'adam', max_iter= 60, batch_size= 128)
classifier.fit(X_train_BERT, y_train)
predictions = classifier.predict(X_test_BERT)
print('accuracy:', accuracy_score(y_test, predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


accuracy: 0.8797086498483854


Which classifier of the three performed better, and why do you think so ?

Ans: MLP performed the best as it is a non-linear classifier, while the support vector machines and naive bayes are linear classifiers. 

#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [14]:
y_train_1 = y_train[:,[0]]
y_train_2 = y_train[:,[1]]
y_train_3 = y_train[:,[2]]
y_train_4 = y_train[:,[3]]
y_train_5 = y_train[:,[4]]
y_train_6 = y_train[:,[5]]

y_test_1 = y_test[:,[0]]
y_test_2 = y_test[:,[1]]
y_test_3 = y_test[:,[2]]
y_test_4 = y_test[:,[3]]
y_test_5 = y_test[:,[4]]
y_test_6 = y_test[:,[5]]

In [19]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed
import math

def loss_func(out,Y):
  return np.mean((out - Y)**2,axis=0)

def sigmoid(x):
  return (1 / (1 + np.exp(np.negative(x))))

def train_and_predict(x_train, y_train, x_test, y_test, method):
    lr = 0.05
    n_samples = len(X_train_USE)
    n_epochs = 5
    batch_size = 100
    
    if method==1:
        W1 = np.random.normal(loc=0.0, scale = 0.01, size = (512,100))
    else:
        W1 = np.random.normal(loc=0.0, scale = 0.01, size = (768,100))

    W2 = np.random.normal(loc=0.0, scale = 0.01, size = (100,1))
  
    for i in range(n_epochs):
        l = []
        j=0
        while j < n_samples:
            X = x_train[j:j+batch_size]
            Y = y_train[j:j+batch_size]
            j=j+batch_size

            out1 = sigmoid(np.dot(X, W1))
            out2 = sigmoid(np.dot(out1, W2))
            l.append(loss_func(out2,Y))
            g_loss = (out2 - Y)

            d_sig_1 = np.multiply(out1, (1-out1))
            d_sig_2 = np.multiply(out2, (1-out2))

            W2_delta = np.dot(out1.T , np.multiply(d_sig_2, g_loss))
            G_delta = np.dot(np.multiply(d_sig_2, g_loss), W2.T)
            W1_delta = np.dot(X.T, np.multiply(d_sig_1, G_delta))
          
            W1 = W1 - lr * W1_delta
            W2 = W2 - lr * W2_delta
        print("Epoch : %d    Loss : %.3f" %(i+1,np.mean(l)))

    print()   
    out1 = sigmoid(np.dot(x_test, W1))
    out2 = sigmoid(np.dot(out1, W2))

    for i in range(len(out2)):
        if out2[i] > 0.5:
            out2[i] = 1
        else:
            out2[i] = 0
    return out2


In [20]:
U1 = train_and_predict(X_train_USE, y_train_1, X_test_USE, y_test_1,1)
U2 = train_and_predict(X_train_USE, y_train_2, X_test_USE, y_test_2,1)
U3 = train_and_predict(X_train_USE, y_train_3, X_test_USE, y_test_3,1)
U4 = train_and_predict(X_train_USE, y_train_4, X_test_USE, y_test_4,1)
U5 = train_and_predict(X_train_USE, y_train_5, X_test_USE, y_test_5,1)
U6 = train_and_predict(X_train_USE, y_train_6, X_test_USE, y_test_6,1)

Epoch : 1    Loss : 0.096
Epoch : 2    Loss : 0.096
Epoch : 3    Loss : 0.096
Epoch : 4    Loss : 0.094
Epoch : 5    Loss : 0.080

Epoch : 1    Loss : 0.010
Epoch : 2    Loss : 0.010
Epoch : 3    Loss : 0.010
Epoch : 4    Loss : 0.010
Epoch : 5    Loss : 0.010

Epoch : 1    Loss : 0.053
Epoch : 2    Loss : 0.053
Epoch : 3    Loss : 0.053
Epoch : 4    Loss : 0.053
Epoch : 5    Loss : 0.053

Epoch : 1    Loss : 0.003
Epoch : 2    Loss : 0.003
Epoch : 3    Loss : 0.003
Epoch : 4    Loss : 0.003
Epoch : 5    Loss : 0.003

Epoch : 1    Loss : 0.049
Epoch : 2    Loss : 0.049
Epoch : 3    Loss : 0.049
Epoch : 4    Loss : 0.049
Epoch : 5    Loss : 0.049

Epoch : 1    Loss : 0.009
Epoch : 2    Loss : 0.009
Epoch : 3    Loss : 0.009
Epoch : 4    Loss : 0.009
Epoch : 5    Loss : 0.009



In [21]:
from sklearn.metrics import accuracy_score
result = np.concatenate((U1, U2, U3, U4, U5, U6), axis=1)
acc= accuracy_score(result, y_test)
print(acc)

0.8987620744630966


In [22]:
B1 = train_and_predict(X_train_BERT, y_train_1, X_test_BERT, y_test_1,2)
B2 = train_and_predict(X_train_BERT, y_train_2, X_test_BERT, y_test_2,2)
B3 = train_and_predict(X_train_BERT, y_train_3, X_test_BERT, y_test_3,2)
B4 = train_and_predict(X_train_BERT, y_train_4, X_test_BERT, y_test_4,2)
B5 = train_and_predict(X_train_BERT, y_train_5, X_test_BERT, y_test_5,2)
B6 = train_and_predict(X_train_BERT, y_train_6, X_test_BERT, y_test_6,2)

Epoch : 1    Loss : 0.096
Epoch : 2    Loss : 0.096
Epoch : 3    Loss : 0.047
Epoch : 4    Loss : 0.036
Epoch : 5    Loss : 0.034

Epoch : 1    Loss : 0.010
Epoch : 2    Loss : 0.010
Epoch : 3    Loss : 0.010
Epoch : 4    Loss : 0.010
Epoch : 5    Loss : 0.010

Epoch : 1    Loss : 0.053
Epoch : 2    Loss : 0.053
Epoch : 3    Loss : 0.053
Epoch : 4    Loss : 0.053
Epoch : 5    Loss : 0.053

Epoch : 1    Loss : 0.003
Epoch : 2    Loss : 0.003
Epoch : 3    Loss : 0.003
Epoch : 4    Loss : 0.003
Epoch : 5    Loss : 0.003

Epoch : 1    Loss : 0.049
Epoch : 2    Loss : 0.049
Epoch : 3    Loss : 0.049
Epoch : 4    Loss : 0.049
Epoch : 5    Loss : 0.049

Epoch : 1    Loss : 0.009
Epoch : 2    Loss : 0.009
Epoch : 3    Loss : 0.009
Epoch : 4    Loss : 0.009
Epoch : 5    Loss : 0.009



In [23]:
from sklearn.metrics import accuracy_score
result = np.concatenate((B1, B2, B3, B4, B5, B6), axis=1)
acc2 = accuracy_score(result, y_test)
print(acc2)

0.882318922129482


Which MLP performed better? 
Understand and Analyse.

Both the MLPs are quite similar in terms of performance. The accuracies vary slightly with changes in batch size, learning rate, and the number of epochs.



